# Langchain, Entity resolution, using tools
this notebook is a companion to [the post](https://medium.com/@mrsirsh/does-your-llm-model-understand-your-entities-3887ebdc37d3)

In [7]:
import json
import pandas as pd
#you have install lanchain and its deps and set your OPEN AI key
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool, BaseTool
#for the vectorstore too
from langchain.vectorstores import LanceDB
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
#pip install this one
import lancedb
import duckdb

LANCE_ROOT = "/tmp/"
DUCK_ROOT = "/tmp/"

#df clean utility
def expand_column_drop(df, column, alias=None, levels=1):
    return expand_column(df, column=column, alias=alias, drop=True, levels=levels)

def expand_column(df, column, drop=False, alias=None, levels=1):
    def check_nan(d):
        if pd.isnull(d):
            return {}
        return d
    if column not in df.columns:
        return df
    alias = alias or column
    _df = pd.DataFrame([check_nan(d) for d in df[column]])
    _df.columns = [f"{alias}_{c}" for c in _df.columns]
    df = df.join(_df)
    if drop:
        df = df.drop(columns=[column], axis=1)
    return df


def tool_from_trivia_df(df, table_name='trivia_store'):
    """
    This tool first loads a lance db table and then creates a QA tool over it
    """
    text_column = 'text'
    # some dataframe that includes a text column
    loader = DataFrameLoader(df, page_content_column=text_column)
    documents = RecursiveCharacterTextSplitter().split_documents(loader.load())
    embeddings = OpenAIEmbeddings()

    def probe(_df):
        d = dict(_df.iloc[0])
        d["vector"] = embeddings.embed_query(d[text_column])
        return d

    db = lancedb.connect(LANCE_ROOT)
    table = db.create_table( table_name, data=[probe(df)], mode="overwrite" )

    docsearch = LanceDB.from_documents(documents, embeddings, connection=table)
    qa = RetrievalQA.from_chain_type(
        llm=ChatOpenAI(model_name="gpt-4", temperature=0.0),
        chain_type="stuff",
        retriever=docsearch.as_retriever(),
    )
    
    return  Tool(
            name="Further details tool",
            func=qa.run,
            description = """If and only if the other tools return no results, use this tool to get extra inforamtion about entities that you are asked about""" )



def ER_tool_from_data(df):
    """
    this is a simple Entity Resolution tool 
    It is basically a wrapper around a dictionary of values loaded from  the sample dataframe
    """
    def fetcher(keys):
        keys = keys.split(',')
        return {k: mapping.get(k.rstrip('\n').lstrip().rstrip()) for k in keys}
    
    mapping = {}
    #based on the format of the dataframe build a map with values like
    #'CAT00': {'legs': 4, 'sound': 'meow', 'size': 'small'},
    #'YELLO': {'rgb': '255,255,0', 'hue': 60, 'complementary': 'BLUE'},
    for record in df.to_dict('records'):
        keys = record['code'].split('-')
        keys_typed = dict(zip(['animal','color', 'shape'],  keys))
        mapping[record['code']] = {
            "code" : record['code'],
            "description" : "this is the code for our friend. The code is made of three components or identifiers, Animal, Color and Shape"
        }
        for key_type, key in keys_typed.items():
            attributes = record[f"{key_type}_attributes"]
            attributes['type'] = key_type
            mapping[key] = attributes

    return  Tool(
            name="Entity Resolution Tool",
            func=fetcher,
            description = """use this tool when you need to lookup entity attributes or find out of some code or identifier.
            Do not use this tool to answer questions of a statistical nature.
            you should pass a comma seprated list of known or suspectedentities to use this tool""" )

def duck_tool_using_text_to_sql_for_df(duck_cursor, table_path, df, enums=None):
    """
    this is a simple tool constructor that takes a dataframe and a connection to duckdb and a file path
    its a text to sql agent that takes a question from the user, requests an SQL statement in duckdb dialect and then executes the query
    the results of this are the tool output for use in chains
    """
    #assume this for now
    llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)
    #wrap the question in the prompt - its the tools job to make a smart prompt
    def ask(question):
        prompt =  f"""For a table called TABLE with the {df.columns}, give me an sql query for duckdb that answers the question {question} """
        query = llm.predict(prompt)
        print(query)
        query = query.replace("TABLE", f"'{table_path}'")
        return duck_cursor.execute(query).fetchdf()
    
    return  Tool(
            name="Stats and data table tool",
            func=ask,
            description = """Use this tool to answer questions about aggregates or statistics or to get sample values or lists of values. Do not select any values that are not in the provided list of columns""" )



# this builds the datesets

In [ ]:

llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)
data =llm.predict("""Please generate a dataset in JSON format with 10 items by combining animals, colors and shapes. 
Each item should have a code determined by the composition of the animal name, color, and shape using upper case. 
When generating the code ensure that each term in the code is exactly 5 letters by zero-padding or truncating the term in the code.  
For example if the animal color and shape are CAT, YELLOW, HEXAGON the code can be CAT00-YELLO-HEXAG.
For each row do the following:
- provide values for 7 "bizarre observations" (numerical, categorical and boolean) such as  "number of times has flow to space" or "believes in aliens" 
- Add 5 sample attributes for each of the animal, color and shape properties
- Use the values 0 and 1 for boolean columns
""")
df = pd.DataFrame([d for d in json.loads(data)])
#if gpt returns a column called data
df = pd.DataFrame(json.loads(data)['data'])


In [4]:
df = expand_column_drop(df, 'bizarre_observations', alias='fact')
df

,animal,color,shape,code,animal_attributes,color_attributes,shape_attributes,fact_times_flown_to_space,fact_believes_in_aliens,fact_number_of_legs,fact_can_swim,fact_favorite_food,fact_has_wings,fact_can_climb_trees
0,CAT,YELLOW,HEXAGON,CAT00-YELLO-HEXAG,"{'fur_length': 'short', 'eye_color': 'green', ...","{'hue': 60, 'saturation': 100, 'lightness': 50...","{'sides': 6, 'interior_angle': 120, 'exterior_...",3,1,4,0,tuna,0,1
1,DOG,GREEN,CIRCLE,DOG00-GREEN-CIRCL,"{'fur_length': 'medium', 'eye_color': 'brown',...","{'hue': 120, 'saturation': 100, 'lightness': 2...","{'sides': 1, 'interior_angle': 'N/A', 'exterio...",0,0,4,1,bone,0,0
2,FISH,BLUE,SQUARE,FISH0-BLUE0-SQUAR,"{'scale_type': 'cycloid', 'eye_color': 'black'...","{'hue': 240, 'saturation': 100, 'lightness': 5...","{'sides': 4, 'interior_angle': 90, 'exterior_a...",0,0,0,1,algae,0,0
3,BIRD,RED,TRIANGLE,BIRD0-RED00-TRIAN,"{'feather_color': 'red', 'eye_color': 'yellow'...","{'hue': 0, 'saturation': 100, 'lightness': 50,...","{'sides': 3, 'interior_angle': 60, 'exterior_a...",0,0,2,0,worm,1,0
4,SNAKE,PURPLE,OCTAGON,SNAKE-PURPL-OCTAG,"{'scale_type': 'keeled', 'eye_color': 'black',...","{'hue': 300, 'saturation': 100, 'lightness': 5...","{'sides': 8, 'interior_angle': 135, 'exterior_...",0,0,0,1,mouse,0,1
5,HORSE,ORANGE,PENTAGON,HORSE-ORANG-PENTA,"{'fur_length': 'short', 'eye_color': 'brown', ...","{'hue': 30, 'saturation': 100, 'lightness': 50...","{'sides': 5, 'interior_angle': 108, 'exterior_...",0,0,4,1,hay,0,0
6,COW,BROWN,RECTANGLE,COW00-BROWN-RECTA,"{'fur_length': 'short', 'eye_color': 'black', ...","{'hue': 30, 'saturation': 100, 'lightness': 20...","{'sides': 4, 'interior_angle': 90, 'exterior_a...",0,0,4,0,grass,0,0
7,SHEEP,WHITE,RHOMBUS,SHEEP-WHITE-RHOMB,"{'fur_length': 'long', 'eye_color': 'blue', 't...","{'hue': 0, 'saturation': 0, 'lightness': 100, ...","{'sides': 4, 'interior_angle': 'variable', 'ex...",0,0,4,0,grass,0,0
8,PIG,PINK,PARALLELOGRAM,PIG00-PINK0-PARAL,"{'fur_length': 'short', 'eye_color': 'black', ...","{'hue': 350, 'saturation': 100, 'lightness': 8...","{'sides': 4, 'interior_angle': 'variable', 'ex...",0,0,4,1,corn,0,0
9,ELEPHANT,GRAY,TRAPEZOID,ELEPH-GRAY0-TRAPE,"{'fur_length': 'short', 'eye_color': 'brown', ...","{'hue': 0, 'saturation': 0, 'lightness': 50, '...","{'sides': 4, 'interior_angle': 'variable', 'ex...",0,0,4,1,leaves,0,0


In [5]:
df.to_csv("./sample.csv")

In [9]:
table_path = f"{DUCK_ROOT}/partition_0.parquet"
 
clean_df = df[[c for c in df.columns if '_attributes' not in c and c not in ['color', 'shape', 'animal']]].rename(columns={"code": "animal_code"})
clean_df.to_parquet(table_path)
duck_cursor = duckdb.connect()
duck_cursor.execute(f"select * from '{table_path}'")

In [10]:
from tqdm import tqdm
from langchain.utilities import WikipediaAPIWrapper
wikipedia = WikipediaAPIWrapper()
trivia = []
for record in tqdm(df.to_dict('records')):
    for c in ['animal', 'color', 'shape']:
        trivia.append(   {"entity_type" : c, "entity_key": record[c], 'text' : wikipedia.run(record[c])})
trivia = pd.DataFrame(trivia).reset_index()
trivia['id'] = trivia['index']
#trivia = trivia.drop('level_0',1)
trivia  

  0%|                                                                                       | 0/10 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
100%|██████████████████████████████████████████████████████████████████████████████| 10/10 [01:45<00:00, 10.51s/it]


,index,entity_type,entity_key,text,id
0,0,animal,CAT,Page: Cat\nSummary: The cat (Felis catus) is a...,0
1,1,color,YELLOW,Page: Yellow\nSummary: Yellow is the color bet...,1
2,2,shape,HEXAGON,"Page: Hexagon\nSummary: In geometry, a hexagon...",2
3,3,animal,DOG,Page: Dog\nSummary: The dog (Canis familiaris ...,3
4,4,color,GREEN,Page: Green\nSummary: Green is the color betwe...,4
5,5,shape,CIRCLE,Page: Circle\nSummary: A circle is a shape con...,5
6,6,animal,FISH,"Page: Fish\nSummary: Fish are aquatic, craniat...",6
7,7,color,BLUE,Page: Blue\nSummary: Blue is one of the three ...,7
8,8,shape,SQUARE,"Page: Square\nSummary: In Euclidean geometry, ...",8
9,9,animal,BIRD,Page: Bird\nSummary: Birds are a group of warm...,9


# Construct the tools

In [13]:
ertool = ER_tool_from_data(df)
stats_tool = duck_tool_using_text_to_sql_for_df(duck_cursor, table_path, clean_df )
trivia_tool = tool_from_trivia_df(trivia)
tools = [ertool,stats_tool, trivia_tool]

# setup the agents

In [14]:
# If you are asked about orders you can use the sku as an identifier. this is needed because the queries that come back are undirected without that e.g. * results so this helps to refine
prefix =  f"""
    Answer the question in the context of any entities you observe in the question or in the context. Follow this strategy:
    - You should typically get context by running the stats and data tool first if you can.
    - To answer the question expand any entity codes into their components, and pass all components to the entity resolution tools.
    - The Further details tool should only be used to augment the context when other tools do not provide an answer.
    Lets takes this step by step.
    You have access to the following tools:"""

suffix = """Begin! Give terse answers.
    Question: {input}
    {agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "agent_scratchpad"],
)
llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)
llm_chain = LLMChain(
    llm=llm, prompt=prompt
)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

# Questions

In [15]:
agent_executor.run("What is the shape angle for the animal that flew to space most often and what sound does it typically make and how many of these animals are there expected to be in the world?")



> Entering new AgentExecutor chain...
Thought: I need to find the animal that flew to space most often.
Action: Stats and data table tool
Action Input: animal, space_flights
SELECT animal_code AS animal, fact_times_flown_to_space AS space_flights
FROM TABLE;

Observation:               animal  space_flights
0  CAT00-YELLO-HEXAG              3
1  DOG00-GREEN-CIRCL              0
2  FISH0-BLUE0-SQUAR              0
3  BIRD0-RED00-TRIAN              0
4  SNAKE-PURPL-OCTAG              0
5  HORSE-ORANG-PENTA              0
6  COW00-BROWN-RECTA              0
7  SHEEP-WHITE-RHOMB              0
8  PIG00-PINK0-PARAL              0
9  ELEPH-GRAY0-TRAPE              0
Thought:The animal that flew to space most often is CAT00-YELLO-HEXAG.
Action: Entity Resolution Tool
Action Input: CAT00-YELLO-HEXAG

Observation: {'CAT00-YELLO-HEXAG\n': {'code': 'CAT00-YELLO-HEXAG', 'description': 'this is the code for our friend. The code is made of three components or identifiers, Animal, Color and Shape'}

'The shape angle for the animal that flew to space most often is 120 degrees. The sound it typically makes is meowing, purring, trilling, hissing, growling, and grunting. There are expected to be around 700 million cats in the world (220 million owned and 480 million stray).'